here we open the file and read it line by line. Using for loop to clean data set( romoving " and \t).
create list of column for each line and clean it.
create records and join cols, separate them with tab.(Note that in a .tsv file, each column is separated by the tab.)
create tsv file and write corresponded records(Store the documents in a directory with inside one file per house review)

In [5]:
file = open("Airbnb_Texas_Rentals.csv",encoding="utf8")
lines = file.readlines()[1:]
for line in lines:
    cols = line.replace('"', '').replace('\t', ' ').split(',')
    record = "\t".join(cols[1:])
    with open('doc\doc_{}.tsv'.format(cols[0]), 'w', encoding="utf8") as file:
            file.write(record)

we need all path of files(docpaths) that created in previous section to open and read them

In [6]:
from os import listdir
from os.path import isfile, join
docpaths = [f for f in listdir("doc") if isfile(join("doc", f))]

we create two function, one for saving the dictionary, and another for load it.
Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

In [7]:
def save_dict_to_file(dic, file):
    with open('{}.txt'.format(file), 'w', encoding="utf-8") as file:
        file.write(str(dic))
    
def load_dict_from_file(file):
    with open('{}.txt'.format(file), 'r', encoding="utf-8") as file:
        data = file.read()
        return eval(data)

As a first common step, you must preprocess the documents by

   Removing stopwords
   Removing punctuation
   Stemming
   Anything else you think it's needed

first we tokenize the documents and for each token repeat this process:
stemmer is for stemming the word

In [10]:
import re
import nltk
#import inflect

#in_engine = inflect.engine()
stemmer = nltk.stem.LancasterStemmer()
def clean_document(document):
    tokens = nltk.word_tokenize(document)
    filtered_words = []
    for word in tokens:
        if word not in nltk.corpus.stopwords.words('english'):#Removing stopwords (list of stop words )
            word = word.lower()
            #if word.isdigit():
            #        word = in_engine.number_to_words(word)
            word = re.sub(r'[^\w\s]', '', word)   #Removing punctuation
            if(len(word) > 0):
                word = stemmer.stem(word)
                filtered_words.append(word)
            
    return filtered_words
#print(clean_document('Now, we want to create two different Search Engines that, given as input a query, return the houses that match the query.'))  

['now', 'want', 'cre', 'two', 'diff', 'search', 'engin', 'giv', 'input', 'query', 'return', 'hous', 'match', 'query']


create vocabulary and inverted index dictionary 



In [5]:
vocabulary = {}
inverted_index = {}
indexOfWord = 0
for doc in docpaths:
    with open('doc\{}'.format(doc), 'r',encoding="utf8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\\n',' ').split('\t')[:-1]
            document = '\t'.join(cols)
            filtered_words = clean_document(document)
            for w in filtered_words:
                if w not in vocabulary:
                    indexOfWord += 1
                    vocabulary[w] = indexOfWord
                if vocabulary[w] not in inverted_index:
                    temp = []
                    temp.append(doc)
                    inverted_index[vocabulary[w]] = temp
                elif doc not in inverted_index[vocabulary[w]]:
                    inverted_index[vocabulary[w]].append(doc)
save_dict_to_file(inverted_index,"inverted_index")
save_dict_to_file(vocabulary,"vocabulary")
print('finish')        


finish


In [11]:
inverted_index = load_dict_from_file('inverted_index')
vocabulary = load_dict_from_file('vocabulary')

In [ ]:
query = input()
query_tokens = clean_document(query)

result_inverted = {}
for token in query_tokens:
    if token in vocabulary:
        word_index = vocabulary[token]
        result_inverted[token] = inverted_index[word_index]
result_set = set()
for item in result_inverted:
    result_set.update(set(result_inverted[item]))

final_set = set(result_set)
for item in result_inverted:
    final_set.intersection_update(set(result_inverted[item]))
print(final_set)

In [4]:
import math
def tf(word, dcoument):
    return dcoument.count(word) / len(dcoument)

def word_containing(word, documentlist):
    return sum(1 for dcoument in documentlist if word in dcoument)

def idf(word, documentlist):
    return math.log(len(documentlist) / (1 + word_containing(word, documentlist)))

def tfidf(word, document, documentlist):
    return tf(word, document) * idf(word, documentlist)

In [29]:
from os import listdir
from os.path import isfile, join
docpaths = [f for f in listdir("doc") if isfile(join("doc", f))]

In [30]:
documentlist = {}
indexOfWord = 0
for doc in docpaths:
    with open('doc\{}'.format(doc), 'r',encoding="utf8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\\n',' ').split('\t')[:-1]
            document = '\t'.join(cols)
            filtered_words = clean_document(document)
            documentlist[doc] = filtered_words

save_dict_to_file(documentlist,"documentlist")


In [5]:
documentlist = load_dict_from_file("documentlist")

In [ ]:
inverted_index = {}
for key,doc in documentlist.items():
    for w in doc:
        score = tfidf(w,doc,documentlist)
        w_index = (key,score)
        if vocabulary[w] not in inverted_index:
            temp = []
            temp.append(w_index)
            inverted_index[vocabulary[w]] = temp
        elif doc not in inverted_index[vocabulary[w]]:
            inverted_index[vocabulary[w]].append(w_index)

save_dict_to_file(inverted_index,"inverted_index_tfidf")
